In [ ]:
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy import stats
from sklearn.impute import KNNImputer
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from pandas.plotting import parallel_coordinates
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso
import pickle
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from xgboost import XGBClassifier
from scipy.stats import uniform, reciprocal
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from pickle import dump,load
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from xgboost import XGBRegressor

In [ ]:
df = pd.read_csv('/content/ApartmentRentPrediction_Milestone2.csv')

**check nulls and redudent records**

In [ ]:
#check dublicates
duplicate_rows = df.duplicated()
redundant_records_count = duplicate_rows.sum()
print(f"redundant records: {redundant_records_count}")

#check nulls
for column in df.columns:
    null_count = df[column].isnull().sum()
    print(f"Null count for column '{column}': {null_count}")

redundant records: 0
Null count for column 'id': 0
Null count for column 'category': 0
Null count for column 'title': 0
Null count for column 'body': 0
Null count for column 'amenities': 3185
Null count for column 'bathrooms': 30
Null count for column 'bedrooms': 7
Null count for column 'currency': 0
Null count for column 'fee': 0
Null count for column 'has_photo': 0
Null count for column 'pets_allowed': 3751
Null count for column 'RentCategory': 0
Null count for column 'price_type': 0
Null count for column 'square_feet': 0
Null count for column 'address': 2971
Null count for column 'cityname': 66
Null count for column 'state': 66
Null count for column 'latitude': 7
Null count for column 'longitude': 7
Null count for column 'source': 0
Null count for column 'time': 0


convert title and body columns to its length

In [ ]:
def body_and_title(df):
  i = 0
  for x in df['title']:
    if isinstance(x,str):
       df.loc[i,'title'] = len(x)
    else:
       df.loc[i,'title'] = 1
    i = i + 1

  i = 0
  for x in df['body']:
    if isinstance(x,str):
       df.loc[i,'body'] = len(x)
    else:
       df.loc[i,'body'] = 1
    i = i + 1
  return df

df = body_and_title(df)

with open('body_and_title.pkl', 'wb') as f:
    pickle.dump(body_and_title, f)


Preprocessing cityname column

In [ ]:
def get_mode_cityname():
  return 'Austin'

def frequency_map(df):
  # frequancy encoding
  frequency_map = df['cityname'].value_counts(normalize=True)
  df['cityname'] = df['cityname'].map(frequency_map)
  return df

df = frequency_map(df)

with open('/content/frequency_map.pkl', 'wb') as f:
     pickle.dump(frequency_map, f)


with open('get_mode_cityname.pkl', 'wb') as f:
    pickle.dump(get_mode_cityname, f)
f.close()

Preprocessing amenities column

In [ ]:
def preprocess_amenities_column(df):
    cnt = -1
    num = 0
    total_sum = 0

    my_set = set()
    my_dict = {}
    for x in df['amenities']:
        cnt = cnt + 1
        if isinstance(x, str):
            s = ""
            for c in x:
                s = s + c
                if c == ',':
                    s = s[:-1]
                    if s in my_dict:
                        my_dict[s] = my_dict[s] + 1
                    else:
                        my_dict[s] = 1
                    my_set.add(s)
                    s = ""
            my_set.add(s)
            if s in my_dict:
                my_dict[s] = my_dict[s] + 1
            else:
                my_dict[s] = 1

    dict_length = len(my_dict)
    my_dict = {k: v for k, v in sorted(my_dict.items(), key=lambda item: item[1])}

    for key in my_dict:
        my_dict[key] = 1 / my_dict[key]





    cnt = -1
    for x in df['amenities']:
        cnt = cnt + 1
        count = 0
        if isinstance(x, str):
            s = ""
            for c in x:
                s = s + c
                if c == ',':
                    s = s[:-1]
                    count = my_dict[s] + count
                    s = ""
            count = my_dict[s] + count
            total_sum = total_sum + count
            num = num + 1
            df.at[cnt, 'amenities'] = count

    mean = total_sum / num
    df['amenities'].fillna(mean, inplace=True)
    return df

df = preprocess_amenities_column(df)


with open('amenitie_function.pkl', 'wb') as f:
    pickle.dump(preprocess_amenities_column, f)
f.close()

In [ ]:
# Perform label encoding based on the condition
def RentCategory_encoding(df):
  # Define the condition
  condition = {'Low Rent': 1, 'Medium-Priced Rent': 2, 'High Rent': 3}
  df['RentCategory'] = df['RentCategory'].map(condition).fillna(0).astype(int)
  return df

# Save the rentcategory encoding
df=RentCategory_encoding(df)

with open('RentCategory_encoding.pkl', 'wb') as f:
    pickle.dump(RentCategory_encoding, f)
f.close()


Address column


In [ ]:
def addressfunc(df):
  cnt = -1
  for x in df['address']:
    cnt = cnt + 1
    if isinstance(x, str):
      df.at[cnt, 'address'] = 1
    else:
      df.at[cnt, 'address'] = 0
  return df
df=addressfunc(df)

with open('addressfunc.pkl', 'wb') as f:
    pickle.dump(addressfunc, f)
f.close()


One Hot Encoding for 'category' , 'price_type' , 'has_photo' , 'fee' ,'currency' , 'RentCategory' columns.


In [ ]:
mode_state_time = 'TX'

def get_mode_state_time():
  return mode_state_time

  #Drop nulls in 'state' column
df.dropna(subset=['state'], inplace=True)

def columns_to_encode(df):
  #Fill the nulls with 'no_pets' in pets_allowed column
  df['pets_allowed'].fillna("NO_PETS", inplace=True)

  print(df['category'].unique())
  print(df['price_type'].unique())
  print(df['has_photo'].unique())
  print(df['fee'].unique())
  print(df['currency'].unique())
  print(df['pets_allowed'].unique())
  print(df['source'].unique())
  print(df['state'].unique())

  columns_to_encode = ['category','price_type','has_photo','fee' , 'currency', 'source' , 'pets_allowed','state']
  # Initialize the OneHotEncoder
  encoder = OneHotEncoder(sparse=False)
  # Fit and transform the data
  encoded_data = encoder.fit_transform(df[columns_to_encode])
  # Create a DataFrame from the encoded data
  encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(columns_to_encode))
  # Concatenate the original DataFrame with the encoded DataFrame
  result_df = pd.concat([df, encoded_df], axis=1)


  #drop the original columns except 'state'
  df = result_df.drop(columns=['category','price_type','has_photo','fee' , 'currency', 'source' , 'pets_allowed'])
  return df

df=columns_to_encode(df)
with open('columns_to_encode.pkl', 'wb') as f:
    pickle.dump(columns_to_encode, f)
f.close()


['housing/rent/apartment' 'housing/rent/short_term' 'housing/rent/home']
['Monthly' 'Weekly' 'Monthly|Weekly']
['Thumbnail' 'Yes' 'No']
['No']
['USD']
['Cats,Dogs' 'NO_PETS' 'Dogs' 'Cats']
['RentDigs.com' 'RentLingo' 'RealRentals' 'ListedBuy' 'Listanza'
 'RENTCafé' 'RENTOCULAR' 'GoSection8' 'tenantcloud' 'Real Estate Agent'
 'rentbits']
['NC' 'WI' 'FL' 'NE' 'CA' 'LA' 'WA' 'OK' 'TX' 'MN' 'VA' 'IN' 'OR' 'OH'
 'NJ' 'PA' 'ND' 'KS' 'IL' 'AK' 'MA' 'AZ' 'SC' 'MD' 'IA' 'CO' 'GA' 'NY'
 'MO' 'TN' 'DC' 'MI' 'NH' 'UT' 'AR' 'CT' 'NV' 'RI' 'AL' 'SD' 'KY' 'VT'
 'NM' 'MS' 'MT' 'ID' 'HI' 'WV' 'DE' 'WY']


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
df = df.dropna()
filter_latitude_state = df[df.state == 'TX']
mean_latitude = filter_latitude_state.latitude.mean()
mean_longitude = filter_latitude_state.longitude.mean()
def get_mean_x_y():
  return mean_latitude , mean_longitude


with open('get_mean_x_y.pkl', 'wb') as f:
      pickle.dump(get_mean_x_y, f)
f.close()


**bathrooms and bedrooms column**

In [ ]:
mean_rooms = 0
def bathroom_bedroom_column(df):
  #Replace float numbers with Integers.
  df['bathrooms'] = df['bathrooms'].astype(int)
  df['bedrooms'] = df['bedrooms'].astype(int)

  #Generate new column "rooms" which is the summetion of bathrooms and bedrooms.

  rooms = df['bathrooms'] + df['bedrooms']
  df.insert(1, 'rooms', rooms)
  mean_rooms = df['rooms'].mean()
  mean_rooms = math.floor(mean_rooms)


  # Generate new column "Studio"  , 1 if studio else 0.
  studio = []
  cnt = 0
  for x in df['bedrooms']:
    if x == 0:
       studio.append(1)
    else:
       studio.append(0)
    cnt = cnt + 1
  df.insert(1, 'studio', studio)

  #Generate new column "room_ratio" = square_feet / rooms.
  new_col = df['square_feet'] / df['rooms']
  df.insert(1, 'room_ratio', new_col)

  #Drop bathrooms and bedrooms
  df = df.drop(['bathrooms'], axis=1)
  df = df.drop(['bedrooms'], axis=1)
  return df
df=bathroom_bedroom_column(df)

# get mean rooms
def get_mean_rooms():
  return math.floor(mean_rooms)

with open('get_mean_rooms.pkl', 'wb') as f:
      pickle.dump(get_mean_rooms, f)

with open('Bathroom_bedroom_column.pkl', 'wb') as f:
      pickle.dump(bathroom_bedroom_column, f)
f.close()


<ipython-input-11-ea41ca943599>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['bathrooms'] = df['bathrooms'].astype(int)
<ipython-input-11-ea41ca943599>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['bedrooms'] = df['bedrooms'].astype(int)


**Change every state with its time zone and do OHE**

In [ ]:
with open('get_mode_state_time.pkl', 'wb') as f:
    pickle.dump(get_mode_state_time, f)
f.close()


def state_time_zones_func(df):
  state_time_zones = {
      'AL': 'UTC-6',
      'AK': 'UTC-9',
      'AZ': 'UTC-7',
      'AR': 'UTC-6',
      'CA': 'UTC-8',
      'CO': 'UTC-7',
      'CT': 'UTC-5',
      'DE': 'UTC-5',
      'FL': 'UTC-5',
      'GA': 'UTC-5',
      'HI': 'UTC-10',
      'ID': 'UTC-7',
      'IL': 'UTC-6',
      'IN': 'UTC-5',
      'IA': 'UTC-6',
      'KS': 'UTC-6',
      'KY': 'UTC-5',
      'LA': 'UTC-6',
      'ME': 'UTC-5',
      'MD': 'UTC-5',
      'MA': 'UTC-5',
      'MI': 'UTC-5',
      'MN': 'UTC-6',
      'MS': 'UTC-6',
      'MO': 'UTC-6',
      'MT': 'UTC-7',
      'NE': 'UTC-6',
      'NV': 'UTC-8',
      'NH': 'UTC-5',
      'NJ': 'UTC-5',
      'NM': 'UTC-7',
      'NY': 'UTC-5',
      'NC': 'UTC-5',
      'ND': 'UTC-6',
      'OH': 'UTC-5',
      'OK': 'UTC-6',
      'OR': 'UTC-8',
      'PA': 'UTC-5',
      'RI': 'UTC-5',
      'SC': 'UTC-5',
      'SD': 'UTC-6',
      'TN': 'UTC-5' ,
      'TX': 'UTC-6',
      'UT': 'UTC-7',
      'VT': 'UTC-5',
      'VA': 'UTC-5',
      'WA': 'UTC-8',
      'WV': 'UTC-5',
      'WI': 'UTC-6',
      'WY': 'UTC-7',
      'DC': 'UTC-5'
  }


  df = df.reset_index(drop=True)
  cnt = 0
  for i in df['state']:
    df.loc[cnt , 'state'] = state_time_zones[i]
    cnt = cnt + 1

  columns_to_encode = ['state']
  encoder = OneHotEncoder(sparse=False)
  encoded_data = encoder.fit_transform(df[columns_to_encode])
  encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(columns_to_encode))
  result_df = pd.concat([df, encoded_df], axis=1)

  df = result_df.drop(columns=columns_to_encode)
  return df

df=state_time_zones_func(df)
with open('state_time_zones.pkl', 'wb') as f:
    pickle.dump(state_time_zones_func, f)
f.close()


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
# Initialize the MinMaxScaler
def minMaxScaler(df):
  scaler = MinMaxScaler()
  columns_to_normalize = ['id','time' , 'title' , 'body','cityname']
  df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])
  return df

df=minMaxScaler(df)
with open('minmaxscaler.pkl', 'wb') as f:
    pickle.dump(minMaxScaler, f)
f.close()


In [ ]:
# Outliers
def remove_outliers_iqr(df, column):
  q1 = df[column].quantile(0.25)
  q3 = df[column].quantile(0.75)
  iqr = q3 - q1
  lower_bound = q1 - 1.5 * iqr
  upper_bound = q3 + 1.5 * iqr
  return df[(df[column] > lower_bound) & (df[column] < upper_bound)]

In [ ]:
# Decoding
# Define the condition
def decodefunc(df):
  condition = { 1:'Low Rent', 2:'Medium-Priced Rent', 3 : 'High Rent'}
  # Perform label encoding based on the condition
  df['RentCategory'] = df['RentCategory'].map(condition).fillna(0).astype(str)
  return df

# Save the decoding of RentCategory
with open('RentCategory_Decode.pkl', 'wb') as f:
    pickle.dump(decodefunc, f)
f.close()


In [ ]:
df = df.drop(['state_UTC-6'], axis=1)
df = df.drop(['state_UTC-10'], axis=1)

def correlationfunc(df):
  df = df.drop(['id'], axis=1)
  df = df.drop(['time'], axis=1)
  df = df.drop(['body'], axis=1)
  df = df.drop(['title'], axis=1)
  df = df.drop(['room_ratio'], axis=1)
  df = df.reset_index(drop=True)
  corr = df.corr()
  top_feature = corr.index[abs(corr['RentCategory']) >= 0.03]
  top_corr = df[top_feature].corr()
  top_feature = top_feature.drop(['RentCategory'])
  return top_feature

top_feature=correlationfunc(df)
df = decodefunc(df)

# Save the Correlation
with open('correlation.pkl', 'wb') as f:
    pickle.dump(correlationfunc, f)
f.close()

print(top_feature)

Index(['rooms', 'square_feet', 'address', 'cityname', 'latitude', 'longitude',
       'state_UTC-8', 'state_UTC-9'],
      dtype='object')


In [ ]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))



SVM


In [ ]:
start_time = timer(None) # timing starts from this point for "start_time" variable

X = df[top_feature]
y = df['RentCategory']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

svm_clf = SVC()
# Define the hyperparameters to tune
param_dist = {
    'C': [0.1, 1, 10 , 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    }

# Randomized search with cross-validation
random_search = RandomizedSearchCV(estimator=svm_clf, param_distributions=param_dist, n_iter=10, cv=5, n_jobs=-1, verbose=2)

# Perform hyperparameter tuning
random_search.fit(X_train, y_train)

# Get the best parameters and best score
best_params = random_search.best_params_
best_score = random_search.best_score_
print("Best Parameters:", best_params)
print("Best Score:", best_score)

# Evaluate the model on the test set using the best parameters
best_svm_clf = SVC(**best_params)
best_svm_clf.fit(X_train, y_train)
y_pred = best_svm_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy on Test Set:",accuracy)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best Parameters: {'gamma': 0.001, 'C': 100}
Best Score: 0.6778938949399944
Accuracy on Test Set: 0.6796830786644029


decision tree classifier


In [ ]:

param_grid = {
    'max_depth': [None, 10, 20, 30]
}

# Initialize the Decision Tree Classifier
dt_classifier = DecisionTreeClassifier(random_state=20)

# Perform grid search
grid_search = GridSearchCV(dt_classifier, param_grid, cv=5, scoring='accuracy', return_train_score=True)
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and their corresponding scores
print("Best Hyperparameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

# Get the best estimator
best_dt_classifier = grid_search.best_estimator_

# Predictions on the test set
dt_pred = best_dt_classifier.predict(X_test)

#Save the best lr classifier using pickle
with open('best_dt_classifier.pkl', 'wb') as f:
    pickle.dump(best_dt_classifier, f)

# Evaluate the classifier
print("Decision Tree Classifier (Tuned):")
print("Accuracy:", accuracy_score(y_test, dt_pred))
print("Classification Report:\n", classification_report(y_test, dt_pred))

Best Hyperparameters: {'max_depth': 10}
Best Score: 0.7417199954754348
Decision Tree Classifier (Tuned):
Accuracy: 0.7402376910016978
Classification Report:
                     precision    recall  f1-score   support

         High Rent       0.77      0.72      0.74       275
          Low Rent       0.69      0.71      0.70       533
Medium-Priced Rent       0.76      0.77      0.76       959

          accuracy                           0.74      1767
         macro avg       0.74      0.73      0.74      1767
      weighted avg       0.74      0.74      0.74      1767



Random Forest


In [ ]:
param_grid = {
    'n_estimators': [10, 50, 200],
    'max_depth': [None, 10, 20]
}

# Initialize the Random Forest Classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Perform grid search
grid_search = GridSearchCV(rf_classifier, param_grid, cv=5, scoring='accuracy', return_train_score=True)
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and their corresponding scores
print("Best Hyperparameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

# Get the best estimator
best_rf_classifier = grid_search.best_estimator_
#Save the best rf classifier using pickle
with open('best_rf_classifier.pkl', 'wb') as f:
    pickle.dump(best_rf_classifier, f)
# Predictions on the test set
rf_pred = best_rf_classifier.predict(X_test)


print("Accuracy:", accuracy_score(y_test, rf_pred))

Best Hyperparameters: {'max_depth': 20, 'n_estimators': 200}
Best Score: 0.7725716247693923
Accuracy: 0.7685342388228636


logistic

In [ ]:
param_grid = {
    'C': [0.1, 1, 10],
    'penalty': ['l2']
}

# Initialize the Logistic Regression model
lr_classifier = LogisticRegression(random_state=42, solver='liblinear', max_iter=10000)

# Perform grid search
grid_search = GridSearchCV(lr_classifier, param_grid, cv=5, scoring='accuracy', return_train_score=True)
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and their corresponding scores
print("Best Hyperparameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

# Get the best estimator
best_lr_classifier = grid_search.best_estimator_
#Save the best lr classifier using pickle
with open('best_lr_classifier.pkl', 'wb') as f:
    pickle.dump(best_lr_classifier, f)
# Predictions on the test set
lr_pred = best_lr_classifier.predict(X_test)

# Evaluate the classifier
print("Logistic Regression Classifier (Tuned):")
print("Accuracy:", accuracy_score(y_test, lr_pred))

Best Hyperparameters: {'C': 10, 'penalty': 'l2'}
Best Score: 0.6450600656061967
Logistic Regression Classifier (Tuned):
Accuracy: 0.6598754951895869


KNN

In [ ]:
# Define the hyperparameter grid
param_grid = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

# Initialize the KNN classifier
knn_classifier = KNeighborsClassifier()

# Perform grid search
grid_search = GridSearchCV(knn_classifier, param_grid, cv=5, scoring='accuracy', return_train_score=True)
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and their corresponding scores
print("Best Hyperparameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

# Get the best estimator
best_knn_classifier = grid_search.best_estimator_

# Predictions on the test set
knn_pred = best_knn_classifier.predict(X_test)
#Save the best KNN classifier using pickle
with open('best_knn_classifier.pkl', 'wb') as f:
    pickle.dump(best_knn_classifier, f)
f.close()
# Evaluate the classifier
print("KNN Classifier (Tuned):")
print("Accuracy:", accuracy_score(y_test, knn_pred))
print("Classification Report:\n", classification_report(y_test, knn_pred))

Best Hyperparameters: {'metric': 'manhattan', 'n_neighbors': 9, 'weights': 'distance'}
Best Score: 0.6729402967594302
KNN Classifier (Tuned):
Accuracy: 0.7119411431805319
Classification Report:
                     precision    recall  f1-score   support

         High Rent       0.67      0.61      0.64       275
          Low Rent       0.71      0.64      0.68       533
Medium-Priced Rent       0.72      0.78      0.75       959

          accuracy                           0.71      1767
         macro avg       0.70      0.68      0.69      1767
      weighted avg       0.71      0.71      0.71      1767



In [ ]:
models = [
    ('rf', RandomForestClassifier(n_estimators=100)),
    ('gb', GradientBoostingClassifier(n_estimators=100)),
    ('knn', KNeighborsClassifier(n_neighbors=5))
]

# Voting Classifier
voting_clf = VotingClassifier(estimators=models, voting='hard')
voting_clf.fit(X_train, y_train)
voting_preds = voting_clf.predict(X_test)
voting_accuracy = accuracy_score(y_test, voting_preds)
print("Voting Classifier Accuracy:", voting_accuracy)

# Stacking Classifier
stacking_clf = StackingClassifier(estimators=models, final_estimator=SVC())
stacking_clf.fit(X_train, y_train)
stacking_preds = stacking_clf.predict(X_test)
stacking_accuracy = accuracy_score(y_test, stacking_preds)
print("Stacking Classifier Accuracy:", stacking_accuracy)

timer(start_time) # timing ends here for "start_time" variable

Voting Classifier Accuracy: 0.7753254102999434
Stacking Classifier Accuracy: 0.7804187889077533

 Time taken: 0 hours 5 minutes and 13.14 seconds.


In [ ]:
def Classifier_test_script(data):
  #Load the frequency map
  with open('/content/frequency_map.pkl', 'rb') as f:
    freq_map = pickle.load(f)


  f1=open('/content/amenitie_function.pkl', 'rb')
  f2=open('RentCategory_encoding.pkl', 'rb')
  f3=open('columns_to_encode.pkl', 'rb')
  f4=open('Bathroom_bedroom_column.pkl', 'rb')
  f5=open('state_time_zones.pkl', 'rb')
  f6=open('minmaxscaler.pkl', 'rb')
  f7=open('correlation.pkl', 'rb')
  f8=open('RentCategory_Decode.pkl', 'rb')
  # f9=open('preprocessed_data.pkl', 'rb')
  f10= open('body_and_title.pkl', 'rb')
  f11= open('addressfunc.pkl', 'rb')
  f12=open('get_mean_rooms.pkl', 'rb')
  f13= open('get_mode_state_time.pkl', 'rb')
  mode_cityname = open('get_mode_cityname.pkl' , 'rb')
  mean_x_y = open('get_mean_x_y.pkl' , 'rb')




  # freq_map = load(f)
  amenitie_function = load(f1)
  RentCategory_encoder = load(f2)
  columns_to_encode =load(f3)
  bathroom_bedroom_column =load(f4)
  state_time_zones =load(f5)
  minMaxScaler = load(f6) #####
  corrrelationfunc = load(f7)
  RentCategory_Decoder =load(f8)
  # df = load(f9)
  body_and_title=load(f10)
  addressfunc = load(f11)
  mean_rooms = load(f12)
  mode_state_time =load(f13)
  get_mode_cityname = load(mode_cityname)
  mean_x_y = load(mean_x_y)

  # implement
  data=body_and_title(data)
  data=amenitie_function(data)
  data=RentCategory_encoding(data)
  data=addressfunc(data)
  data['bathrooms'].fillna(mean_rooms() , inplace=True)
  data['bedrooms'].fillna(0 , inplace=True)
  data=bathroom_bedroom_column(data)
  data['cityname'].fillna(get_mode_cityname() , inplace=True)
  data = freq_map(data)
  data['state'].fillna(mode_state_time() , inplace=True)
  mean_latitude , mean_longitude = mean_x_y()
  data['latitude'].fillna(mean_latitude , inplace=True)
  data['longitude'].fillna(mean_longitude , inplace=True)
  data=columns_to_encode(data)
  data=state_time_zones_func(data)
  data=minMaxScaler(data)
  data=decodefunc(data)


  y = data['RentCategory']
  data = data[top_feature]

  return data , y

In [ ]:
# Read the new CSV file
new_df = pd.read_csv('/content/ApartmentRentPrediction_classification_test.csv')
X_new_data , y = Classifier_test_script(new_df)
X_train, X_test, y_train, y_test = train_test_split(X_new_data, y, test_size=0.2, random_state=42)

['housing/rent/apartment' 'housing/rent/home']
['Monthly']
['Yes' 'Thumbnail' 'No']
['No']
['USD']
['NO_PETS' 'Cats,Dogs' 'Cats' 'Dogs']
['RentDigs.com' 'RentLingo' 'ListedBuy' 'GoSection8' 'RealRentals'
 'Listanza' 'rentbits' 'RENTOCULAR' 'Home Rentals']
['NC' 'NJ' 'OH' 'OK' 'CO' 'WI' 'CA' 'VA' 'TX' 'UT' 'OR' 'AZ' 'CT' 'MD'
 'GA' 'IN' 'PA' 'MO' 'WA' 'ME' 'ND' 'FL' 'MS' 'NV' 'MA' 'KS' 'ID' 'NE'
 'IL' 'IA' 'MI' 'MN' 'SD' 'NH' 'DC' 'LA' 'SC' 'NM' 'NY' 'AL' 'TN' 'RI'
 'AR' 'HI' 'VT' 'AK']


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
# Load the best models
with open('best_dt_classifier.pkl', 'rb') as f:
    best_dt_classifier = pickle.load(f)
f.close()


with open('best_rf_classifier.pkl', 'rb') as f:
    best_rf_classifier = pickle.load(f)
f.close()


with open('best_lr_classifier.pkl', 'rb') as f:
    best_lr_classifier = pickle.load(f)
f.close()


with open('best_knn_classifier.pkl', 'rb') as f:
    best_knn_classifier = pickle.load(f)
f.close()


In [ ]:
# Make predictions using the best models
dt_pred = best_dt_classifier.predict(X_test)
rf_pred = best_rf_classifier.predict(X_test)
lr_pred = best_lr_classifier.predict(X_test)
knn_pred = best_knn_classifier.predict(X_test)

In [ ]:
# Evaluate the classifiers
print("Decision Tree Classifier :")
print("Accuracy:", accuracy_score(y_test, dt_pred))

print("Random Forest Classifier :")
print("Accuracy:", accuracy_score(y_test, rf_pred))

print("Logistic Regression Classifier :")
print("Accuracy:", accuracy_score(y_test, lr_pred))

print("KNN Classifier :")
print("Accuracy:", accuracy_score(y_test, knn_pred))

Decision Tree Classifier :
Accuracy: 0.67
Random Forest Classifier :
Accuracy: 0.71
Logistic Regression Classifier :
Accuracy: 0.615
KNN Classifier :
Accuracy: 0.665
